# 标识、相等性和别名
## 标识
每个变量都有标识、类型和值，对象一旦创建，它的标识最对象的生命周期内不会改变，id()返回对象的标识（CPython中就是内存地址的整数标识）。
## 相等性
==运算符（\__eq__的语法糖）比较对象的值，is比较标识。默认创建的类的\__eq__继承自object，其实现就是比较两个对象的ID。is运算符由于不可重载因此调用速度更快。
## 别名
引用相对于对象来说就是标签，不同的‘标签’其实就是对象的不同名称。
# 元组的不可变性
元组的不可变性是值tuple数据结构的物理内容不可变性，与如果元组含有引用，则该引用不可变，与该引用指定的对象无关。因此虽然元组是不可变类型，但若其中含有对可变对象的引用，则该元组不可散列。
# 默认浅复制行为
内置List的构造方法和[:]做的是浅复制，即赋值了最外层的容器对象，但是副本中引用的元素是原容器中对应的对象。

In [52]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1)
print(l2 == l1)
print(l2 is l1)
# list使用+=就地修改(使用__iadd__，相当于append或extend。
l2[1] += [33]
# l2中的列表改变，l1中也发生改变
print(l1)
# l2中元组是不可变对象，使用+=（使用__add__）后返回新的元组对象，因此l1中的元组不受影响
l2[-1] += (10,)
print(l1)
print(l2)

True
False
[3, [55, 44, 33], (7, 8, 9)]
[3, [55, 44, 33], (7, 8, 9)]
[3, [55, 44, 33], (7, 8, 9, 10)]


## 做深拷贝浅拷贝
copy模块提供copy()和deepcopy()为任意对象提供浅拷贝和深拷贝

In [53]:
class Bus:
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
            
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)
        
import copy

bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)
bus1.drop('Bill')
print(bus2.passengers)
# bus3是深拷贝，因此有独立的passengers列表,bus1、bus2拥有的passengers是同一个list对象
print(bus3.passengers)
print(id(bus1.passengers), id(bus2.passengers), id(bus3.passengers))

['Alice', 'Claire', 'David']
['Alice', 'Bill', 'Claire', 'David']
2075452375816 2075452375816 2075452278536


In [58]:
# 面对循环引用时，deepcopy也能正常工作，因为它会记住已经复制过的对象，当再次遇到后只需要将其引用进行传递即可。
a = [10, 20]
b = [a, 30]
a.append(b)
c = copy.deepcopy(a)
c

[10, 20, [[...], 30]]

# 向函数传递引用以及可变类型作为参数默认值的问题
向函数传递引用，函数的形参实际上是实参的别名，因此函数可能会修改作为参数传入的可变对象，但无法修改这些对象的标识。
#### 在定义函数的默认值时应当避免使用可变类型，默认值在定义函数时计算（通常在加载模块时）变成了函数对象的属性，因此若默认值是可变对象且对其进行了修改，那么后续函数被调用时使用的默认值都会受到影响。

In [63]:
class Bus:
    def __init__(self, passengers=[]):
        self.passengers = passengers
            
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)
        
import copy

bus1 = Bus()
bus1.pick('Bill')
bus2 =  Bus()
# 由于bus1使用了默认的空[]且增加了Bill，因此再次使用默认值时原先的空[]变为['Bill']
print(bus2.passengers)
bus2.pick('David')
print(bus1.passengers)
# 函数默认值已发生改变
print(Bus.__init__.__defaults__[0])

['Bill']
['Bill', 'David']
['Bill', 'David']


# 谨慎对待可变参数
有时函数接受可变参数后会在函数内部对其进行修改，那么被传入的对象是否希望被修改是需要考虑的，如果不希望就需要使用传入的可变对象的引用来对原对象做一个拷贝，基于拷贝来进行计算或修改，从而不影响原对象。
# del和垃圾回收
del删除的是引用而不是对象本身。del如果恰好删除了某个对象的最后一个可达引用（除了两个对象互相引用这样的引用）会使得该对象的引用计数为0，导致对象被立即销毁。Cpython2.0中还增加了分代垃圾回收算法用于检测引用循环中的对象组。


如下代码在CPthron2.0中是安全代码:

In [ ]:
open('test.txt', 'wt', encoding='utf8').write('1, 2, 3')  

这段代码存在对文件对象的引用，当write方法返回后，该引用的计数归0使得文件对象立刻被回收并关闭相关资源。但是在Jython和IronPython中并不安全，因为它们使用的是宿主的运行时垃圾回收器（Java VM和.NET CLR），这些垃圾回收器并不以引用计数为基准因此销毁对象和关闭文件的时间可能更长。因此在任何情况下都应该显式的关闭文件。

关闭文件最可靠的方式是使用with语句，即便打开文件时抛出了异常也一定能保证文件被关闭：

In [ ]:
with open('test.txt', 'wt', encoding='utf8') as fp:  
    fp.write('1, 2, 3')

# 弱引用
当有引用只想对象时，对象才会在内存中存在，但有些时候我们想引用一个对象又不想使得其作为该对象的引用计数而影响对象的回收，这时便可使用弱引用。weakref.ref可返回对象的弱引用。
#### 很多情况下常常存在不明显的隐式引用，当使用python控制台时，会自动把_变量绑定给不为None的执行结果上，此时就可能导致相关对象未被回收但常常不知道哪里还存在引用。

In [82]:
import weakref
a_set = {0, 1}
wref = weakref.ref(a_set)
print(wref)
# 弱引用调用时（wref()）返回被引用的对象。
print(wref())
# {0, 1}唯一引用被重置，因此计数为0，被回收。
a_set = {2, 3}
# 弱引用指向的对象不存在了，返回None。
print(wref())

<weakref at 0x000001E33A9AC908; to 'set' at 0x000001E33A9B4048>
{0, 1}
None


In [90]:
import weakref
s1 = {1, 2, 3}
s2 = s1
def bye():
    print('Gone')
# 在s1所指对象上绑定回调函数bye，这里得到的监控对象ender内包含对s1所指对象的弱引用。
ender = weakref.finalize(s1, bye)
print(ender.alive)
del s1
print(ender.alive)
s2 = 'spam'
print(ender.alive)

True
True
Gone
False


# WeakValueDictionary
WeakValueDictionary是一种可变映射，其值是对象的弱引用，当值对应的对象被回收后，映射中对应的键值对会被删除。WeakValueDictionary常用于缓存。

In [100]:
import weakref

class Cheese:
    def __init__(self, kind):
        self.kind = kind
        
    def __repr__(self):
        return 'Cheese(%r)' % self.kind
    
stock = weakref.WeakValueDictionary()
catalog = [Cheese('Red Leicester'), Cheese('Tilsit'), Cheese('Brie'), Cheese('Parmesan')]
for cheese in catalog:
    stock[cheese.kind] = cheese
    
print(sorted(stock.keys()))
# 删除list的唯一引用catalog，list被销毁（内部的全部引用元素被销毁）
del catalog
# Cheese('Parmesan')对象仍然存在引用指向它，其实是for循环中的cheese变量，而且该变量是全局变量，因此尽量不要定义全局变量影响垃圾回收。
print(sorted(stock.keys()))

['Brie', 'Parmesan', 'Red Leicester', 'Tilsit']
['Parmesan']


与WeakValueDictionary对应的是WeakKeyDictionary，其键是对象的弱引用。该类可用于为对象（可散列）添加额外数据，该数据随着对象的回收而回收。  
另外weakref还提供WeakSet，用于保存元素的弱引用集合。它可以用于保存某个类所有的类的实例。
## 弱引用的局限
CPython作为解释器的python中不是所有的对象都可以作为弱引用的目标，基本的list和dict实例就不行，但是其任意子类却可以。另外任何用户自定义类型也没有问题但是int和tuple及其子类的实例都不可以成为弱引用的目标。这些行为时CPython解释器内部优化所导致的结果。
# python对不可变类型的优化
1.对于元组来说，[:]不创建副本，且利用构造函数得到的仍然是同一个元组，但利用字面量创建的对象是不同的。

2.frozenset.copy()、str、bytes同样返回同一个对象的引用，另外frozenset不是序列不能使用[]运算。

3.相同对象字面量的str仍然是同一个对象。这种共享字符串对象的方式是一种优化措施，称为__驻留__，可以避免大量创建相同的对象。类似的在小的整数上也会采用这种方案__防止重复创建常用值__。
#### 这种对不可变类型的优化其实对用户没有任何影响，因为对于不可变对象来说，相等（==）的对象是同一个对象还是不同的对象对于用户来说没有区别。

In [106]:
t1 = (1, 2, 3)
t2 = tuple(t1)
t3 = t1[:]
t4 = (1, 2, 3)
print(t1 is t2)
print(t1 is t3)
print(t1 is t4)

True
True
False


In [109]:
a1 = str('abc')
a2 = str('abc')
a3 = 'abc'
print(a1 is a2)
print(a1 is a3)

True
True


In [110]:
c1 = int(1)
c2 = 1
print(c1 is c2)

True
